In [1]:
# Affichage de la carte avec tous les groupes de facteurs

In [2]:
import os
import pandas as pd
import geopandas as gpd
import folium
from branca.element import MacroElement
from jinja2 import Template

In [3]:

# ---------- Chargement des données ----------
base_dir = "/home/oneai/jumeau_num"  # chemin

if not os.path.exists(base_dir):
    raise FileNotFoundError(f"Le dossier {base_dir} n'existe pas !")

gdf_com_2024_path = os.path.join(base_dir, "gdf_com_2024.geojson")
gdf_dep_path = os.path.join(base_dir, "gdf_dep.geojson")
factors_path = os.path.join(base_dir, "tables", "facteurs_en_groupes.csv")

gdf_com_2024 = gpd.read_file(gdf_com_2024_path)
gdf_dep = gpd.read_file(gdf_dep_path)
factors = pd.read_csv(factors_path)

In [4]:
# ---------- Fonction de classification ----------
def classify_factors(df, exclude_cols=["codgeo", "cluster", "libgeo", "canton"]):
    """
    Classe les colonnes numériques d'un DataFrame en niveaux (0 à 4)
    en fonction des quartiles de leur distribution.
    - 0 : valeur nulle
    - 1 : <= Q1
    - 2 : <= Q2 (médiane)
    - 3 : <= Q3
    - 4 : > Q3
    - None : valeurs manquantes
    """
    df_classified = df.copy()
    num_cols = df_classified.select_dtypes(include="number").columns
    num_cols = [c for c in num_cols if c not in exclude_cols]

    for col in num_cols:
        q1 = df_classified[col].quantile(0.25)
        q2 = df_classified[col].quantile(0.50)
        q3 = df_classified[col].quantile(0.75)

        def classify(val):
            if pd.isna(val):
                return None
            elif val == 0:
                return 0
            elif val <= q1:
                return 1
            elif val <= q2:
                return 2
            elif val <= q3:
                return 3
            else:
                return 4

        df_classified[col] = df_classified[col].apply(classify)

    return df_classified

In [5]:
# ---------- Classification des facteurs par commune ----------
factors_classified = classify_factors(factors)

# ---------- Fusion des données brutes par commune ----------
factors_classified['codgeo'] = factors_classified['codgeo'].astype(str)
gdf_com_2024['COM_geojson'] = gdf_com_2024['COM_geojson'].astype(str)

gdf_factors_com = gdf_com_2024.merge(factors_classified, left_on='COM_geojson', right_on='codgeo')

# ---------- Ajout de la colonne 'canton' dans gdf_com_2024 avant dissolution ----------
gdf_com_2024 = gdf_com_2024.merge(
    factors[['codgeo', 'canton']],
    left_on='COM_geojson',
    right_on='codgeo',
    how='left'
)

# ---------- Moyenne des facteurs par canton ----------
factor_cols = [col for col in factors.columns if col not in ['codgeo', 'libgeo', 'cluster', 'canton']]
factors_canton = factors.groupby('canton')[factor_cols].mean().reset_index()

# ---------- Classification des facteurs moyens par canton ----------
factors_canton_classified = classify_factors(factors_canton, exclude_cols=['canton'])

# ---------- Fusion géométrique des communes par canton ----------
gdf_canton_geom = gdf_com_2024.dissolve(by='canton', as_index=False)

# ---------- Fusion des facteurs classifiés avec la géométrie des cantons ----------
gdf_canton_factors = gdf_canton_geom.merge(factors_canton_classified, on='canton')

In [ ]:
# suppression cluster
if 'cluster' in factors.columns:
    factors = factors.drop(columns=['cluster'])

if 'cluster' in factors_classified.columns:
    factors_classified = factors_classified.drop(columns=['cluster'])

if 'cluster' in factors_canton_classified.columns:
    factors_canton_classified = factors_canton_classified.drop(columns=['cluster'])


In [6]:
# ---------- Définition des couleurs des niveaux ----------
niveau_couleurs = {
    1: "#ffffb2",  # jaune clair
    2: "#fecc5c",  # orange clair
    3: "#fd8d3c",  # orange foncé
    4: "#e31a1c"   # rouge
}

In [7]:
# ---------- Création de la carte ----------
carte_france = folium.Map(
    location=[46.5, 2],
    zoom_start=6,
    min_zoom=5.9,
    max_bounds=True,
    tiles="CartoDB positron"
)

bounds_sud_ouest = [41.0, -5.0]
bounds_nord_est = [51.5, 9.5]
carte_france.fit_bounds([bounds_sud_ouest, bounds_nord_est])
carte_france.options['maxBounds'] = [bounds_sud_ouest, bounds_nord_est]

# Couche départements
folium.GeoJson(
    gdf_dep,
    name="Départements",
    style_function=lambda x: {"color": "black", "weight": 0.5, "fillOpacity": 0},
    tooltip=folium.GeoJsonTooltip(fields=["nom", "code"], aliases=["Département", "Code INSEE"])
).add_to(carte_france)

# Couche communes (bords seulement)
folium.GeoJson(
    gdf_com_2024,
    name="Communes",
    style_function=lambda x: {"color": "blue", "weight": 0.1, "fillOpacity": 0.2},
    tooltip=folium.GeoJsonTooltip(fields=["DCOE_L_LIB", "COM_geojson"], aliases=["Commune", "Code INSEE"])
).add_to(carte_france)

In [8]:
# ---------- Fonction style pour communes ----------
def make_style_function_commune(col):
    def style_function(feature):
        value = feature["properties"].get(col)
        if value == 0 or pd.isna(value):
            return {
                "fillColor": "white",
                "color": "transparent",
                "weight": 0.2,
                "fillOpacity": 0.6
            }
        color = niveau_couleurs.get(value, "transparent")
        return {
            "fillColor": color,
            "color": "black",
            "weight": 0.2,
            "fillOpacity": 0.6
        }
    return style_function

# Ajout couches communes classifiées
for col in factor_cols:
    fg_com = folium.FeatureGroup(name=f"Commune: {col}", show=False)
    folium.GeoJson(
        gdf_factors_com,
        name=col,
        style_function=make_style_function_commune(col),
        tooltip=folium.GeoJsonTooltip(
            fields=["COM_geojson", col],
            aliases=["Commune", f"Facteur: {col}"]
        )
    ).add_to(fg_com)
    fg_com.add_to(carte_france)

In [9]:
# ---------- Fonction style pour cantons ----------
def make_style_function_canton(col):
    def style_function(feature):
        value = feature["properties"].get(col)
        if value == 0 or pd.isna(value):
            return {
                "fillColor": "white",
                "color": "transparent",
                "weight": 0.5,
                "fillOpacity": 0.5
            }
        color = niveau_couleurs.get(value, "transparent")
        return {
            "fillColor": color,
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.5
        }
    return style_function

# Ajout couches cantons classifiés
for col in factor_cols:
    fg_canton = folium.FeatureGroup(name=f"Canton: {col}", show=False)
    folium.GeoJson(
        gdf_canton_factors,
        name=col,
        style_function=make_style_function_canton(col),
        tooltip=folium.GeoJsonTooltip(
            fields=["canton", col],
            aliases=["Canton", f"Facteur: {col}"]
        )
    ).add_to(fg_canton)
    fg_canton.add_to(carte_france)

In [10]:
# ---------- Légende ----------
legend_html = """
{% macro html(this, kwargs) %}
<div style="
bottom: 30px;
left: 30px;
width: 220px;
background-color: white;
border:2px solid grey;
z-index:9999;
font-size:14px;
padding: 10px;
box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
<b>Légende des niveaux</b><br>
<i style="background:#ffffb2;width:12px;height:12px;display:inline-block;"></i>&nbsp; Niveau 1<br>
<i style="background:#fecc5c;width:12px;height:12px;display:inline-block;"></i>&nbsp; Niveau 2<br>
<i style="background:#fd8d3c;width:12px;height:12px;display:inline-block;"></i>&nbsp; Niveau 3<br>
<i style="background:#e31a1c;width:12px;height:12px;display:inline-block;"></i>&nbsp; Niveau 4<br>
</div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
carte_france.get_root().add_child(legend)

In [11]:

# Contrôle des couches
folium.LayerControl(collapsed=True).add_to(carte_france)

# ---------- Sauvegarde et affichage ----------
carte_france.save("carte_france_facteurs_communes_cantons.html")

# Affichage dans notebook Jupyter (optionnel)
from IPython.display import IFrame
IFrame("carte_france_facteurs_communes_cantons.html", width=950, height=600)
